In [1]:
import os
import sys

# Add the Darknet directory to the system path
darknet_path = os.path.abspath('../darknet')
if darknet_path not in sys.path:
    sys.path.append(darknet_path)

# Change the working directory to the Darknet directory
os.chdir(darknet_path)

# Verify the current working directory and Darknet installation
print(f"Current working directory: {os.getcwd()}")
!ls  # List contents of Darknet directory
!./darknet  # Run Darknet to see usage information

# Change back to the notebook's directory
os.chdir(os.path.dirname(os.path.abspath('__file__')))
print(f"Changed back to notebook directory: {os.getcwd()}")

Current working directory: /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet
3rdparty		darknet_video.py       net_cam_v4.sh
backup			data		       obj
bad.list		docker-compose.yml     package.xml
build			Dockerfile.cpu	       README.md
build.ps1		Dockerfile.gpu	       results
cfg			image_yolov3.sh        scripts
cmake			image_yolov4.sh        src
CMakeLists.txt		include		       vcpkg.json
darknet			json_mjpeg_streams.sh  video_yolov3.sh
DarknetConfig.cmake.in	LICENSE		       video_yolov4.sh
darknet_images.py	Makefile	       yolov3-tiny.weights
darknet.py		net_cam_v3.sh	       yolov3-tiny.weights.1
usage: ./darknet <function>
Changed back to notebook directory: /home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet


In [2]:
# Download pre-trained weights for the convolutional layers
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2024-10-17 16:30:59--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights.2’

yolov3-tiny.weights 100%[===================>]  33.79M  3.37MB/s    in 16s     

2024-10-17 16:31:59 (2.13 MB/s) - ‘yolov3-tiny.weights.2’ saved [35434956/35434956]



In [3]:
# Create necessary configuration files
with open('data/rumex.data', 'w') as f:
    f.write(f"classes = 1\n")
    f.write(f"train = data/train.txt\n")
    f.write(f"valid = data/valid.txt\n")
    f.write(f"names = data/rumex.names\n")
    f.write(f"backup = backup/\n")

print("Contents of data/rumex.data:")
!cat data/rumex.data

with open('data/rumex.names', 'w') as f:
    f.write("rumex_acetosa\n")

print("Contents of data/rumex.names:")
!cat data/rumex.names

Contents of data/rumex.data:
classes = 1
train = data/train.txt
valid = data/valid.txt
names = data/rumex.names
backup = backup/
Contents of data/rumex.names:
rumex_acetosa


In [4]:
import os
import glob

def create_list(path, output_file):
    darknet_dir = "/home/akash/My_Projects/Optical-Plant-Identification-for-Farming-Vehicles-/darknet"
    with open(output_file, 'w') as f:
        for img_path in glob.glob(os.path.join(path, '*.jpg')):
            # Create path relative to darknet directory
            relative_path = os.path.relpath(img_path, start=darknet_dir)
            f.write(relative_path + '\n')
    print(f"Created {output_file} with {len(glob.glob(os.path.join(path, '*.jpg')))} entries")

# Paths relative to the script location
train_path = '../data/augmented_dataset/train'
valid_path = '../data/augmented_dataset/valid'
train_output = '../darknet/data/train.txt'
valid_output = '../darknet/data/valid.txt'

create_list(train_path, train_output)
create_list(valid_path, valid_output)

print("First few lines of data/train.txt:")
!head -n 5 data/train.txt

print("First few lines of data/valid.txt:")
!head -n 5 data/valid.txt

Created ../darknet/data/train.txt with 250 entries
Created ../darknet/data/valid.txt with 15 entries
First few lines of data/train.txt:
../data/augmented_dataset/train/SAM_1883_JPG.rf.6ac03c86430f7077b6ad7770375674a4_aug_81d96be44e584d2e975d0f57405f32c7.jpg
../data/augmented_dataset/train/SAM_1947_JPG.rf.ab793c5998755603f8ae666cf1e46aea_aug_e6708d235a21461b8cc957dca7bbb947.jpg
../data/augmented_dataset/train/SAM_1912_JPG.rf.e4d8dea398b6cc5bbd5bd4e21104bde4_aug_de53ef651cb34edcb08ce06839e79c0a.jpg
../data/augmented_dataset/train/SAM_1893_JPG.rf.c68d42fee12c9edaa49220c27462d40d_aug_2f9e5ca20efa46caa4c455eb6a8da1a6.jpg
../data/augmented_dataset/train/SAM_1880_JPG.rf.dfc7414e9efb61f33bbd7d70e965a246_aug_c9055c10e35e47249ba18b9d0f8109a5.jpg
First few lines of data/valid.txt:
../data/augmented_dataset/valid/SAM_1895_JPG.rf.34ffcfdc4d452b1d4445cf9584eb58d7.jpg
../data/augmented_dataset/valid/SAM_1921_JPG.rf.2266e7416f743c397fe91f5f108bb462.jpg
../data/augmented_dataset/valid/SAM_1940_JPG.rf.c

In [10]:
# Modify yolov3-tiny configuration
with open('cfg/yolov3-tiny.cfg', 'r') as f:
    config = f.read()

# Adjust batch and subdivisions
config = config.replace('batch=1', 'batch=16')
config = config.replace('subdivisions=1', 'subdivisions=4')

# Adjust for single class
config = config.replace('classes=80', 'classes=1')

# Adjust filters for single class (3 * (5 + classes))
config = config.replace('filters=255', 'filters=18', 2)  # Replace only the last two occurrences

# Adjust max_batches, steps, and learning rate
lines = config.split('\n')
for i, line in enumerate(lines):
    if line.startswith('max_batches'):
        lines[i] = 'max_batches = 4000'  # 2000 * number of classes, but minimum 4000
    elif line.startswith('steps'):
        lines[i] = 'steps=3200,3600'  # 80% and 90% of max_batches
    elif line.startswith('learning_rate'):
        lines[i] = 'learning_rate=0.001'  # Reduced learning rate for fine-tuning

# Add data augmentation options
lines.insert(3, 'angle=0')
lines.insert(4, 'saturation = 1.5')
lines.insert(5, 'exposure = 1.5')
lines.insert(6, 'hue = .1')

config = '\n'.join(lines)

with open('cfg/yolov3-tiny-rumex.cfg', 'w') as f:
    f.write(config)

print("Contents of cfg/yolov3-tiny-rumex.cfg:")
!head -n 30 cfg/yolov3-tiny-rumex.cfg

Contents of cfg/yolov3-tiny-rumex.cfg:
[net]
# Testing
batch=16
subdivisions=4
# Training
# batch=64
# subdivisions=2
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1


In [11]:
import subprocess
import sys

# Start training with transfer learning
try:
    process = subprocess.Popen(
        ['./darknet', 'detector', 'train', 
         'data/rumex.data', 
         'cfg/yolov3-tiny-rumex.cfg', 
         'yolov3-tiny.weights',  # Use the pre-trained weights
         '-map',  # Calculate mAP during training
         '-dont_show'  # Don't show images during training (useful for remote servers)
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True
    )

    print("Training started. Output:")
    for line in process.stdout:
        print(line, end='')
        sys.stdout.flush()  # Ensure output is displayed immediately

    process.wait()
    print("Training process finished with return code:", process.returncode)

except Exception as e:
    print(f"An error occurred: {e}")

Training started. Output:
 CUDA-version: 12000 (12040), cuDNN: 9.4.0, GPU count: 1  
 OpenCV version: 4.6.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: NVIDIA GeForce GTX 1660 Ti 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max                

In [16]:
print("Contents of train.txt:")
!head -n 5 data/train.txt

print("\nContents of valid.txt:")
!head -n 5 data/valid.txt


print("Contents of yolov3-tiny-rumex.cfg:")
!head -n 20 cfg/yolov3-tiny-rumex.cfg


print("Contents of rumex.data:")
!cat data/rumex.data


import cv2
import random

with open('data/train.txt', 'r') as f:
    image_paths = f.readlines()

for _ in range(5):  # Test 5 random images
    path = random.choice(image_paths).strip()
    img = cv2.imread(path)
    if img is None:
        print(f"Failed to load: {path}")
    else:
        print(f"Successfully loaded: {path}")

Contents of train.txt:
../data/augmented_dataset/train/SAM_1883_JPG.rf.6ac03c86430f7077b6ad7770375674a4_aug_81d96be44e584d2e975d0f57405f32c7.jpg
../data/augmented_dataset/train/SAM_1947_JPG.rf.ab793c5998755603f8ae666cf1e46aea_aug_e6708d235a21461b8cc957dca7bbb947.jpg
../data/augmented_dataset/train/SAM_1912_JPG.rf.e4d8dea398b6cc5bbd5bd4e21104bde4_aug_de53ef651cb34edcb08ce06839e79c0a.jpg
../data/augmented_dataset/train/SAM_1893_JPG.rf.c68d42fee12c9edaa49220c27462d40d_aug_2f9e5ca20efa46caa4c455eb6a8da1a6.jpg
../data/augmented_dataset/train/SAM_1880_JPG.rf.dfc7414e9efb61f33bbd7d70e965a246_aug_c9055c10e35e47249ba18b9d0f8109a5.jpg

Contents of valid.txt:
../data/augmented_dataset/valid/SAM_1895_JPG.rf.34ffcfdc4d452b1d4445cf9584eb58d7.jpg
../data/augmented_dataset/valid/SAM_1921_JPG.rf.2266e7416f743c397fe91f5f108bb462.jpg
../data/augmented_dataset/valid/SAM_1940_JPG.rf.c2f3e3b28914c9c7d569e02d82f90d61.jpg
../data/augmented_dataset/valid/SAM_1909_JPG.rf.1de8e0dda6773a35820b4912c53b4a9a.jpg
../

In [7]:
import cv2
import random

with open('data/train.txt', 'r') as f:
    image_paths = f.readlines()

for _ in range(5):  # Test 5 random images
    path = random.choice(image_paths).strip()
    img = cv2.imread(path)
    if img is None:
        print(f"Failed to load: {path}")
    else:
        print(f"Successfully loaded: {path}")

Successfully loaded: ../data/augmented_dataset/train/SAM_1864_JPG.rf.d9dd294b2c10b9a4001fb548def1c738_aug_aa9fdf3f3f6f4d8e96435fef6ab48583.jpg
Successfully loaded: ../data/augmented_dataset/train/SAM_1886_JPG.rf.e2460e1a7eaebd5ea9db9d5c1a285829_aug_3e9a095c50ab48c9bae19d8e35b2a54c.jpg
Successfully loaded: ../data/augmented_dataset/train/SAM_1941_JPG.rf.6434cfbabb3948979930e9688a963ee9_aug_d6e7565e1ffc41938b36e254a1d4a3ba.jpg
Successfully loaded: ../data/augmented_dataset/train/SAM_1916_JPG.rf.e0d5f4c1d35d9fbbde544a298688f6c6_aug_e15fd957ac0d4f8aaa7a8dc202738dd0.jpg
Successfully loaded: ../data/augmented_dataset/train/SAM_1880_JPG.rf.dfc7414e9efb61f33bbd7d70e965a246_aug_8be0fe1103304aac80f80e2238a5c639.jpg
